In [1]:
import pandas as pd

In [2]:
pd.read_pickle('./../../test_noising_experiment.pickle')

,graph_id,target_node,truth,standard_prediction,path_length_0_prediction,path_length_1_prediction,path_length_2_prediction,path_length_3_prediction,path_length_4_prediction,path_length_5_prediction,...,path_length_11_prediction,path_length_12_prediction,path_length_13_prediction,path_length_14_prediction,path_length_15_prediction,path_length_16_prediction,path_length_17_prediction,path_length_18_prediction,path_length_19_prediction,path_length_20_prediction
0,0,0,tensor(0),tensor(0),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0,1,tensor(0),tensor(0),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0,2,tensor(0),tensor(0),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
0,0,3,tensor(0),tensor(0),0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
0,0,4,tensor(0),tensor(0),0.0,8.0,8.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN
